In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

from prophet import Prophet


In [10]:
df_raw = pd.read_csv('./data/copper.csv', parse_dates=['date'])
display(df_raw.head())
display(df_raw.info())

,date,LME Copper Cash-Settlement,LME Copper 3-month,LME Copper stock
0,2024-01-05,8346.5,8450.0,163.175
1,2024-01-04,8375.0,8470.5,164.450
2,2024-01-03,8381.0,8482.0,165.700
3,2024-01-02,8430.0,8539.5,165.700
4,2023-12-29,8476.0,8580.0,167.300


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3794 entries, 0 to 3793
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date                        3794 non-null   datetime64[ns]
 1   LME Copper Cash-Settlement  3794 non-null   float64       
 2   LME Copper 3-month          3794 non-null   float64       
 3   LME Copper stock            3794 non-null   float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 118.7 KB


None

In [242]:
df = df_raw.copy()[['date','LME Copper 3-month']]
df.rename(columns={'date':'ds', df.columns.to_list()[1]:'y'}, inplace=True)

df.dropna(inplace=True)

df = df.set_index('ds')
df = df.asfreq(freq='D')
df = df.interpolate(method='linear')

df.reset_index(inplace=True)
df = df.sort_values(by='ds', ascending=False).reset_index(drop=True)

display(df.head())
display(df.tail())

,ds,y
0,2024-01-05,8450.000
1,2024-01-04,8470.500
2,2024-01-03,8482.000
3,2024-01-02,8539.500
4,2024-01-01,8549.625


,ds,y
5477,2009-01-06,3370.000000
5478,2009-01-05,3160.000000
5479,2009-01-04,3150.333333
5480,2009-01-03,3140.666667
5481,2009-01-02,3131.000000


In [243]:
y_test = df[(df["ds"].dt.year == 2024)]

y_train = df.drop(index=y_test.index).reset_index(drop=True)

display(y_test)
print(len(y_test))
display(y_train.head())

,ds,y
0,2024-01-05,8450.000
1,2024-01-04,8470.500
2,2024-01-03,8482.000
3,2024-01-02,8539.500
4,2024-01-01,8549.625


5


,ds,y
0,2023-12-31,8559.750
1,2023-12-30,8569.875
2,2023-12-29,8580.000
3,2023-12-28,8630.000
4,2023-12-27,8615.000


In [235]:
df = df[df["ds"].dt.year != 2024].reset_index(drop=True)

y_test = df[(df["ds"].dt.year == 2023) & (df["ds"].dt.month > 9)]

y_train = df.drop(index=y_test.index).reset_index(drop=True)

display(y_test)
print(len(y_test))
display(y_train.head())

,ds,y
0,2023-12-31,8559.750000
1,2023-12-30,8569.875000
2,2023-12-29,8580.000000
3,2023-12-28,8630.000000
4,2023-12-27,8615.000000
...,...,...
87,2023-10-05,7890.000000
88,2023-10-04,7986.500000
89,2023-10-03,8030.000000
90,2023-10-02,8172.000000


92


,ds,y
0,2023-09-30,8245.333333
1,2023-09-29,8282.000000
2,2023-09-28,8168.500000
3,2023-09-27,8075.000000
4,2023-09-26,8131.000000


In [236]:
# Улучшите модель добавлением данных о праздниках. Отобразите результат, сделайте письменный вывод.
df_holidays_events = pd.read_csv('./data/holidays_events.csv')
holidays = df_holidays_events[df_holidays_events['transferred'] == False][['description', 'date']]
holidays.columns = ['holiday', 'ds']
display(holidays.tail())

,holiday,ds
345,Navidad-3,2017-12-22
346,Navidad-2,2017-12-23
347,Navidad-1,2017-12-24
348,Navidad,2017-12-25
349,Navidad+1,2017-12-26


In [244]:
m = Prophet(changepoint_prior_scale=2.75,)# holidays=holidays)
m.add_seasonality(name='monthly', period=30.5, fourier_order=10, mode='multiplicative')

m.fit(y_train)

future = m.make_future_dataframe(len(y_test))
prediction = m.predict(future)

y_hat = prediction[["ds","yhat"]].iloc[-len(y_test): ]

y_hat =  y_hat.sort_values(by='ds', ascending=False)

y_hat.reset_index(drop=True, inplace=True)

10:48:37 - cmdstanpy - INFO - Chain [1] start processing
10:48:39 - cmdstanpy - INFO - Chain [1] done processing


In [245]:
y_test

,ds,y
0,2024-01-05,8450.000
1,2024-01-04,8470.500
2,2024-01-03,8482.000
3,2024-01-02,8539.500
4,2024-01-01,8549.625


In [246]:
len(y_hat)

5

In [247]:
mean_absolute_error(y_test['y'], y_hat['yhat'])

101.71563960177664